Problem definition: https://adventofcode.com/2018/day/7

# Part 1

In [28]:
class Node:
    def __init__(self, key):
        self.key = key
        self.succ = []
        self.pred = []

In [29]:
def build_graph(filename):
    f = open(filename, 'r')
    nodes = {}
    for line in f:
        parts = line.split(' ')
        orig_key = parts[1]
        dest_key = parts[7]
        
        if orig_key in nodes:
            orig = nodes[orig_key]            
        else:
            orig = Node(orig_key)
            
        if dest_key in nodes:
            dest = nodes[dest_key]            
        else:
            dest = Node(dest_key)
        
        orig.succ.append(dest)
        dest.pred.append(orig)
        
        if orig_key not in nodes:
            nodes[orig_key] = orig
        
        if dest_key not in nodes:
            nodes[dest_key] = dest
    
    # pre-calculate node traversal order here by sorting successors list
    for k, node in nodes.items():
        node.succ = sorted(node.succ, key = lambda n: n.key, reverse=True)
    return nodes

This problem is solved with a topological sort algorithm (https://en.wikipedia.org/wiki/Topological_sorting). I incidentally implemented something similar to the Kahn's algorithm but without modifying the original graph by removing its edges. Instead I use a visited set data structure to know if the node can be visited or not.

In [30]:
def topological_sort(nodes):
    available = find_available(nodes)

    stack = available
    visited = set()
    order = []
    while stack:
        cur_node = stack.pop()
        visited.add(cur_node)
        order.append(cur_node)

        for s in cur_node.succ:
            if s not in visited and set(s.pred).issubset(visited):
                append_lexicographically(stack, s)

    if len(order) == len(nodes):
        return [node.key for node in order]
    return []

def find_available(nodes):
    available = []
    for k, node in nodes.items():
        if not node.pred:
            available.append(node)
    return sorted(available, key=lambda c: c.key, reverse=True)

def append_lexicographically(stack, s):
    stack2 = []
    while stack and s.key > stack[-1].key:
        stack2.append(stack.pop())
    stack.append(s)
    while stack2:
        stack.append(stack2.pop())

In [31]:
nodes = build_graph('example1.txt')

In [32]:
topological_sort(nodes)

['C', 'A', 'B', 'D', 'F', 'E']

In [33]:
nodes = build_graph('input.txt')
order = topological_sort(nodes)
''.join(order)

'AHJDBEMNFQUPVXGCTYLWZKSROI'

After implementing my solution I found out that it could be solved in just two lines of code.

In [34]:
from networkx import DiGraph, lexicographical_topological_sort as lt_sort

print(''.join(lt_sort(DiGraph((line.split()[1], line.split()[-3]) for line in open('input.txt', 'r')))))

AHJDBEMNFQUPVXGCTYLWZKSROI


# Part 2

Part 2 is just a slight modification of the topological_sort method:
* Keep track of the available workers
* Store the remaining amount of time for each task being executed in parallel, and update it on each iteration.

In [38]:
def calculate_completion_time(nodes, num_workers, task_time):
    available = find_available(nodes) 
    stack = available
    visited = set()
    order = []
    available_workers = num_workers
    cur_nodes = {}
    tic = 0
    while len(order) != len(nodes):
        while available_workers > 0 and stack:            
            node = stack.pop()
            cur_nodes[node] = task_time + ord(node.key) - ord('A') + 1
            available_workers -= 1
        
        tic += 1
        completed_buffer = []
        for node, _ in cur_nodes.items():
            cur_nodes[node] -= 1
            if cur_nodes[node] == 0:
                available_workers = min(num_workers, available_workers + 1)
                visited.add(node)
                order.append(node)

                for s in node.succ:
                    if s not in visited and set(s.pred).issubset(visited):
                        append_lexigrophically(stack, s)
                
                completed_buffer.append(node)
                
        for node in completed_buffer:
            del cur_nodes[node]

    return tic


In [39]:
nodes = build_graph('input.txt')

In [43]:
calculate_completion_time(nodes, 5, 60)

1031